# Lab 1: Hello, Streamable HTTP (MCP)

**Goal:**  
In this lab, you'll run a minimal **FastMCP** server over the **Streamable HTTP** transport, connect to it with a Python MCP client, and call a simple `add(a, b)` tool.

**What you'll learn:**
1. How to start a FastMCP server using the Streamable HTTP transport.
2. How to connect to that server with the official MCP Python client.
3. How to list available tools and call them.
4. How Streamable HTTP differs from STDIO.

**Steps:**
1. Install required Python packages (`fastmcp`, `mcp`).
2. Create and start the MCP server (`server.py`).
3. Connect to the server using the Python client.
4. Call the `add(a, b)` tool and validate results.
5. Compare STDIO vs Streamable HTTP in a short reflection.

**Before you start:**
- This lab runs in **Udemy's JupyterLab 4** environment.
- Your workspace is pre-loaded with this notebook and a blank `server.py` file.
- Run each cell **in order**. If a cell fails, fix the error and re-run it before moving on.
- Use `Shift+Enter` to run the current cell.

**Reflection Prompt:**
At the end, note t


In [1]:
%pip install -q fastmcp "mcp[cli]"


Note: you may need to restart the kernel to use updated packages.


In [2]:
# server_cell.py (inside a notebook cell)
import asyncio
from pydantic import BaseModel
from mcp.server.fastmcp import FastMCP

mcp = FastMCP(stateless_http=True)

class SumResult(BaseModel):
    value: int

@mcp.tool("Add")
def add(a: int, b: int) -> SumResult:
    """Add two integers and return their sum."""
    return SumResult(value=a + b)

# Start the server in the background so the cell returns
server_task = asyncio.create_task(mcp.run_streamable_http_async())
print("MCP server starting…")


MCP server starting…


INFO:     Started server process [28198]
INFO:     Waiting for application startup.


[08/11/25 07:09:06] INFO     StreamableHTTP session manager started                  ]8;id=355591;file:///Users/aref/miniconda3/lib/python3.11/site-packages/mcp/server/streamable_http_manager.py\streamable_http_manager.py]8;;\:]8;id=895160;file:///Users/aref/miniconda3/lib/python3.11/site-packages/mcp/server/streamable_http_manager.py#112\112]8;;\

INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:58713 - "POST /mcp/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:58714 - "POST /mcp/ HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:58715 - "POST /mcp/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:58716 - "POST /mcp/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:58730 - "POST /mcp/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:58731 - "POST /mcp/ HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:58732 - "POST /mcp/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:58733 - "POST /mcp/ HTTP/1.1" 200 OK


In [4]:
# CLIENT CELL
import asyncio
from contextlib import AsyncExitStack
from mcp.client.streamable_http import streamablehttp_client
from mcp import ClientSession

SERVER_URL = "http://127.0.0.1:8000/mcp/"

async def run_client():
    async with AsyncExitStack() as stack:
        values = await stack.enter_async_context(streamablehttp_client(SERVER_URL))
        if isinstance(values, tuple):
            if len(values) == 2:
                read, write = values
            else:
                read, write = values[0], values[1]
        else:
            read, write = values

        session = await stack.enter_async_context(ClientSession(read, write))
        await session.initialize()

        tools = await session.list_tools()
        print("Tools:", [t.name for t in tools.tools])

        res = await session.call_tool("Add", {"a": 40, "b": 2})
        print("Raw result:", res)


        import json
        val = int(json.loads(res.content[0].text)["value"])


        print("Add(40,2) =", val)

        assert val == 42, f"Expected 42, got {val}"
        print("✅ add(40,2) returned 42")

await run_client()


[08/11/25 07:10:58] INFO     HTTP Request: POST http://127.0.0.1:8000/mcp/ "HTTP/1.1 200 OK"        ]8;id=732479;file:///Users/aref/miniconda3/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=266531;file:///Users/aref/miniconda3/lib/python3.11/site-packages/httpx/_client.py#1740\1740]8;;\

                    INFO     Negotiated protocol version: 2025-06-18                         ]8;id=503165;file:///Users/aref/miniconda3/lib/python3.11/site-packages/mcp/client/streamable_http.py\streamable_http.py]8;;\:]8;id=711176;file:///Users/aref/miniconda3/lib/python3.11/site-packages/mcp/client/streamable_http.py#146\146]8;;\

                    INFO     HTTP Request: POST http://127.0.0.1:8000/mcp/ "HTTP/1.1 202 Accepted"  ]8;id=246593;file:///Users/aref/miniconda3/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=186969;file:///Users/aref/miniconda3/lib/python3.11/site-packages/httpx/_client.py#1740\1740]8;;\

                    INFO     Processing request of type ListToolsRequest                              ]8;id=305761;file:///Users/aref/miniconda3/lib/python3.11/site-packages/mcp/server/lowlevel/server.py\server.py]8;;\:]8;id=588985;file:///Users/aref/miniconda3/lib/python3.11/site-packages/mcp/server/lowlevel/server.py#619\619]8;;\

                    INFO     HTTP Request: POST http://127.0.0.1:8000/mcp/ "HTTP/1.1 200 OK"        ]8;id=663975;file:///Users/aref/miniconda3/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=324098;file:///Users/aref/miniconda3/lib/python3.11/site-packages/httpx/_client.py#1740\1740]8;;\

Tools: ['Add']


[08/11/25 07:10:59] INFO     Processing request of type CallToolRequest                               ]8;id=740666;file:///Users/aref/miniconda3/lib/python3.11/site-packages/mcp/server/lowlevel/server.py\server.py]8;;\:]8;id=970771;file:///Users/aref/miniconda3/lib/python3.11/site-packages/mcp/server/lowlevel/server.py#619\619]8;;\

                    INFO     HTTP Request: POST http://127.0.0.1:8000/mcp/ "HTTP/1.1 200 OK"        ]8;id=9365;file:///Users/aref/miniconda3/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=311379;file:///Users/aref/miniconda3/lib/python3.11/site-packages/httpx/_client.py#1740\1740]8;;\

Raw result: meta=None content=[TextContent(type='text', text='{\n  "value": 42\n}', annotations=None, meta=None)] structuredContent={'value': 42} isError=False
Add(40,2) = 42
✅ add(40,2) returned 42
